In [1]:
import wandb
import matplotlib.pyplot as plt
import scipy
import numpy as np
import json

plt.rcParams["axes.grid"] = False

sys.path.append('..')

In [2]:
language = 'french'

sys.argv = [
"--device cuda",
"--data-folder", "..\\data",
"--seed", "13",
"--configuration", "char-to-char",
"--language", language,
"--challenge", "post-ocr-correction"]

In [3]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [4]:
plot_service = container.plot_service()

In [5]:
entity = 'eval-historical-texts'
project = 'named-entity-recognition'

In [18]:
def get_wandb_runs():
    api = wandb.Api()
    runs = api.runs(path=f'{entity}/{project}', filters={
        'createdAt': {
            '$gt': '20200515000000'
        },
        'config.language': language,
        'config.split_documents': True,
        'config.fine_tune_pretrained': False,
        'config.include_pretrained_model': True,
        'state': 'finished'
    })

    return runs

In [19]:
runs = get_wandb_runs()
print(f'Loaded {len(runs)} runs')

Loaded 1 runs


In [21]:
result_dict = {}

for i, run in enumerate(runs):
    score_key = 'Best - f1-score-micro-partial-all-literal-coarse'
    if len(run.summary.keys()) == 0 or run.summary['Epoch'] == 0 or score_key not in run.summary.keys():
        continue

    current_entity_tag_types = len(run.config['entity_tag_types'])
    f1_micro_score = run.summary[score_key]

    if current_entity_tag_types not in result_dict.keys():
        result_dict[current_entity_tag_types] = []

    result_dict[current_entity_tag_types].append(f1_micro_score)

for tasks_amount, f1_scores in result_dict.items():
    print(f'{tasks_amount} tasks\n{round(np.mean(f1_scores), 4)}\n(max: {round(max(f1_scores), 4)}, std: {round(np.std(f1_scores), 6)})')

1 tasks
0.8087
(max: 0.8087, std: 0.0)
